# Train model

In [ ]:
%cd /content/gdrive/MyDrive
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="RyZweMUNaHCXeg9yceug")
project = rf.workspace("ali-wntbu").project("license-plate-ccqf1")
dataset = project.version(1).download("yolov7")

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to License-plate-1 in yolov7pytorch:: 100%|██████████| 8648/8648 [00:02<00:00, 3376.89it/s]


In [ ]:
%%bash
wget -P /content/yolov7 https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 40 --data /content/yolov7/License-plate-1/data.yaml --weights 'yolov7.pt'

YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='/content/yolov7/License-plate-1/data.yaml', device='', entity=None, epochs=40, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, v5_metric=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-07 16:56:00.716345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow b

# Load the model and get predicted plate

In [1]:
import fileinput
import os
from pathlib import Path
from typing import Union
import torch
import cv2 as cv
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
def prepend_text(filename: Union[str, Path], text: str):
    with fileinput.input(filename, inplace=True) as file:
        for line in file:
            if file.isfirstline():
                print(text)
            print(line, end="")
            
if not os.path.isdir('yolov7'):
    yolov7_repo_url = 'https://github.com/WongKinYiu/yolov7'
    os.system(f'git clone {yolov7_repo_url}')
    # Fix import errors
    for file in ['yolov7/models/common.py', 'yolov7/models/experimental.py', 'yolov7/models/yolo.py', 'yolov7/utils/datasets.py']:
         prepend_text(file, "import sys\nsys.path.insert(0, './yolov7')")

In [3]:
%cd /content/yolov7

/content/yolov7


In [4]:
from yolov7.models.experimental import attempt_load
from yolov7.utils.general import check_img_size
from yolov7.utils.torch_utils import select_device, TracedModel
from yolov7.utils.datasets import letterbox
from yolov7.utils.general import non_max_suppression, scale_coords
from yolov7.utils.plots import plot_one_box

In [ ]:
weights = '/content/drive/MyDrive/best.pt'
device_id = 'cpu'
image_size = 640
trace = True

# Initialize
device = select_device(device_id)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(image_size, s=stride)  # check img_size

if trace:
    model = TracedModel(model, device, image_size)

if half:
    model.half()  # to FP16
    
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

model.eval()

In [35]:
def detect_plate(source_image):
    # Padded resize
    img_size = 640
    stride = 32
    img = letterbox(source_image, img_size, stride=stride)[0]

    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
        
    with torch.no_grad():
        # Inference
        pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.25, 0.45, classes=0, agnostic=True)

    plate_detections = []
    det_confidences = []
    
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], source_image.shape).round()

            # Return results
            for *xyxy, conf, cls in reversed(det):
                coords = [int(position) for position in (torch.tensor(xyxy).view(1, 4)).tolist()[0]]
                plate_detections.append(coords)
                det_confidences.append(conf.item())

    return plate_detections, det_confidences

def crop(image, coord):
    cropped_image = image[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
    return cropped_image

def draw_box_img(source_image, plate_detections):
    for coords in plate_detections:
        cv.rectangle(source_image, (coords[0], coords[1]), (coords[2], coords[3]), (0,255,0), 2)
    cv.imwrite("/content/detected_image.jpg", source_image)

In [7]:
source_image_path = "/content/drive/MyDrive/datasets/License-plate-1/test/images/191_-129-_png.rf.e5fc8f0ef650931ba230112a349ab44a.jpg"    
source_image = cv.imread(source_image_path)
print(source_image.shape)
# Padded resize
img_size = 480
stride = 32
img = letterbox(source_image, img_size, stride=stride)[0]

(480, 480, 3)


In [ ]:
plate_detections, det_confidences = detect_plate(img)
print(f'confidence score: {det_confidences}')
draw_box_img(source_image, plate_detections)
plate = crop(source_image, plate_detections[0])
gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)

In [34]:
cv.imwrite("/content/plate.jpg", plate)

True

# OCR

In [10]:
# Apply Gaussian blurring and thresholding 
# to reveal the characters on the license plate
blurred = cv.GaussianBlur(gray, (5, 5), 0)
thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_MEAN_C,
                               cv.THRESH_BINARY_INV, 45, 15)

In [11]:
# Perform connected components analysis on the thresholded image and
# initialize the mask to hold only the components we are interested in
_, labels = cv.connectedComponents(thresh)
mask = np.zeros(thresh.shape, dtype="uint8")

In [12]:
# Set lower bound and upper bound criteria for characters
total_pixels = plate.shape[0] * plate.shape[1]
lower = total_pixels // 70 # heuristic param, can be fine tuned if necessary
upper = total_pixels // 20 # heuristic param, can be fine tuned if necessary

In [13]:
# Loop over the unique components
for (i, label) in enumerate(np.unique(labels)):
    # If this is the background label, ignore it
    if label == 0:
        continue
 
    # Otherwise, construct the label mask to display only connected component
    # for the current label
    labelMask = np.zeros(thresh.shape, dtype="uint8")
    labelMask[labels == label] = 255
    numPixels = cv.countNonZero(labelMask)
 
    # If the number of pixels in the component is between lower bound and upper bound, 
    # add it to our mask
    if numPixels > lower and numPixels < upper:
        mask = cv.add(mask, labelMask)

In [14]:
# Find contours and get bounding box for each contour
cnts, _ = cv.findContours(mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
boundingBoxes = [cv.boundingRect(c) for c in cnts]

In [15]:
import functools
# Sort the bounding boxes from left to right, top to bottom
# sort by Y first, and then sort by X if Ys are similar
def compare(rect1, rect2):
    if abs(rect1[1] - rect2[1]) > 10:
        return rect1[1] - rect2[1]
    else:
        return rect1[0] - rect2[0]
boundingBoxes = sorted(boundingBoxes, key=functools.cmp_to_key(compare))

In [31]:
classes = {
0:	'0',
1:	'1',
2:	'2',
3:	'3',
4:	'4',
5:	'5',
6:	'6',
7:	'7',
8:	'8',
9:	'9',
10:	'الف',
11:	'ب',	
12:	'پ',	
13:	'ج',	
14:	'ه',	
15:	'د',	
16:	'س',	
17:	'ص',	
18:	'ط',	
19: 'ق',
20:	'ل',	
21:	'م',	
22:	'ن',	
23:	'و',	
24:	'ی',
25:	'ع',	
26:	'ت',
27: 'PwD'
}

In [33]:
from tensorflow import keras
keras_model = keras.models.load_model('/content/char_pred.h5')

plate_num = []
for bbox in boundingBoxes:
    # get char from the plate
    x, y, w, h = bbox
    bbox = [x, y, x+w, y+h]
    char_img = crop(plate, bbox)
    # resize to 64*64
    char_img = cv.resize(char_img, (64, 64))
    # predict keras model
    char_img = np.expand_dims(char_img, axis=0)
    pred = keras_model([char_img])
    idx = np.argsort(pred)[0]
    plate_num.append(classes[idx[-1]])

for num in plate_num:
  print(num)

5
6
پ
9
5
3
7
ق


# Train Keras Model

train keras model for char detection

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   rotation_range=5,  
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
                  '/content/train',
                  target_size=(64, 64),
                  batch_size=32,
                  class_mode='categorical', 
                  subset='training')

val_generator = train_datagen.flow_from_directory(
                  '/content/train',
                  target_size=(64, 64),
                  batch_size=32,
                  class_mode='categorical', 
                  subset='validation')

Found 67085 images belonging to 28 classes.
Found 16759 images belonging to 28 classes.


In [ ]:
# Define Your Model HERE!
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(units=len(classes), activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        36896     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10816)             0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(train_generator,
          epochs=10,
          validation_data=val_generator)

Epoch 1/10
2097/2097 [==============================] - 171s 79ms/step - loss: 0.7654 - acc: 0.8031 - val_loss: 0.3690 - val_acc: 0.8949
Epoch 2/10
2097/2097 [==============================] - 158s 75ms/step - loss: 0.1797 - acc: 0.9508 - val_loss: 0.2987 - val_acc: 0.9251
Epoch 3/10
2097/2097 [==============================] - 157s 75ms/step - loss: 0.1270 - acc: 0.9660 - val_loss: 0.2139 - val_acc: 0.9468
Epoch 4/10
2097/2097 [==============================] - 155s 74ms/step - loss: 0.0979 - acc: 0.9742 - val_loss: 0.2231 - val_acc: 0.9505
Epoch 5/10
2097/2097 [==============================] - 157s 75ms/step - loss: 0.0860 - acc: 0.9772 - val_loss: 0.2820 - val_acc: 0.9488
Epoch 6/10
2097/2097 [==============================] - 154s 74ms/step - loss: 0.0780 - acc: 0.9789 - val_loss: 0.2659 - val_acc: 0.9423
Epoch 7/10
2097/2097 [==============================] - 168s 80ms/step - loss: 0.0597 - acc: 0.9846 - val_loss: 0.2687 - val_acc: 0.9539
Epoch 8/10
2097/2097 [===================

In [ ]:
model.save('/content/char_pred.h5')